# GCC's ssc-cdi : tomography pipeline for Cateretê v0.1
 
- #### Questions? E-mail gcc@lnls.br
- #### Installation and User guide [available here](https://10.10.10.141/ssc/wiki/). 
- #### Documentation about the ssc-cdi package: https://gcc.lnls.br/ssc/ssc-cdi/index.html
- #### To use a test sample:
    - Ibira Datapath (str) = /ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho3d/
    - Ibira Datafolders (list) = ['phantom_complex']
    - Ptycho sinogram path (str) = /ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_complex/object_phantom_complex.npy

## Imports

In [1]:
%matplotlib widget
import numpy as np, matplotlib.pyplot as plt
from sscCdi import read_data, define_paths, select_contrast, tomo_sort, tomo_crop, tomo_unwrap, tomo_equalize, preview_angle_projection, tomo_alignment, tomo_recon, tomo_equalize3D, deploy_visualizer

## Load dictionary

In [2]:
dic = {} # or load from template file

## Load data

In [3]:
dic["sinogram_path"] = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_complex/phantom_complex.hdf5"
dic["recon_method"]  = "ptycho" # ptycho or pwcdi
dic["contrast_type"] = "phase" # phase or magnitude
dic = define_paths(dic)
obj, angles = read_data(dic)
print(f"Object shape = {obj.shape} \t Number of angles: {angles.shape}")

Object shape = (128, 64, 64) 	 Number of angles: (128, 2)


## Select contrast

In [4]:
phase_object = select_contrast(dic, obj)
# deploy_visualizer(phase_object,title='Original sinogram')

## Sort data

In [5]:
dic = tomo_sort(dic,phase_object, angles)
# deploy_visualizer(np.load(dic["ordered_object_filepath"]),title="Ordered sinogram")

Time elapsed: 0.00 s


## Crop

In [6]:
dic["top_crop"]    = 15
dic["bottom_crop"] = 1
dic["left_crop"]   = 0
dic["right_crop"]  = 10
dic = tomo_crop(dic)
# deploy_visualizer(np.load(dic["cropped_sinogram_filepath"]),title="Cropped sinogram")

Cropped sinogram shape: (128, 48, 54)
Time elapsed: 0.01 s


## Unwrap

In [7]:
dic["bad_frames_before_unwrap"] = []
dic = tomo_unwrap(dic)
# deploy_visualizer(np.load(dic["unwrapped_sinogram_filepath"]),title="Unwrapped sinogram")

Using 32 parallel processes


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:00<00:00, 4712.20it/s]

Time elapsed: 0.75 s


## 2D Equalization

In [8]:
dic["bad_frames_before_equalization"] = []
dic["equalize_gradient"] = 0
dic["equalize_global_offset"] = False
dic["equalize_invert"] = False
dic["equalize_local_offset"] = [ 0, 10, 0, 10 ]
dic["equalize_outliers"] = 0

dic = tomo_equalize(dic)
deploy_visualizer(np.load(dic["equalized_sinogram_filepath"]),title="Equalized sinogram")

Sinogram shape to unwrap:  (128, 48, 54)
Using 32 parallel processes


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:00<00:00, 5080.88it/s]

Min 	 Mean-3*sigma 	 Mean 	 Mean+3*sigma 	 Max 
Old  0.0 -1.6899914145469666 0.2645012 2.218993842601776 2.931003
New  0.0 -2.108442723751068 0.37819195 2.8648266196250916 8.512125
Time elapsed: 0.27 s


## Alignment

In [9]:
dic["wiggle_sinogram_selection"] = dic["equalized_sinogram_filepath"]
dic["step_percentage"] = 1 # need to project irregular angle steps to a regular grid?

preview_angle_projection(dic)

Simulating projection of angles to regular grid...
Before+after frames added: 7149
Intermediate null frames : 0
                        + -----
Total null frames        : 7149
Frames being used        : 128  of  128
                        + -----
Projected Angles         : 7277


In [10]:
dic["CPUs"] = 32
dic["bad_frames_before_wiggle"] = [  0,  1,   2 ]
dic["wiggle_reference_frame"] = 10

dic = tomo_alignment(dic)
deploy_visualizer(np.load(dic["unwrapped_sinogram_filepath"]),title="Aligned sinogram",cmap='gray',axis=0)

ssc-radon: creating 127x48x54 shared arrays (tomogram)
ssc-radon: creating 127x48x54 shared arrays (tomogram)
	Finished vertical wiggle. Starting horizontal wiggle...
ssc-radon: creating 127x48x54 shared arrays (tomogram)
Time elapsed: 0.49 s


## Tomography

In [11]:
dic["GPUs"] = [0,1,2]
dic["tomo_algorithm"] = "EEM"
dic["tomo_iterations"] = 40
dic["tomo_regularization"] = False
dic["tomo_regularization_param"] = 0.001

dic = tomo_recon(dic)

Starting tomographic algorithm:  EEM
EM param: 20 [40, 3, 8] 0.0001 1e-15 eEM 127 [0, 1, 2] 54 48
ssc-raft: Error! Please check block size!
	Applying wiggle center-of-mass correction to 3D recon slices...
ssc-radon: creating 48x54x54 shared arrays (tomogram)
		 Correction done!
	 Tomography done!
Saving tomography logfile...


KeyError: 'datetime'

## 3D Equalization

In [ ]:
dic["tomo_remove_outliers"] = 0
dic["tomo_threshold"] = 10.0
dic["tomo_local_offset"] = []
dic["tomo_mask"] = []

dic = tomo_equalize3D(dic)

## Save output metadata

In [2]:
# save dic